This Model is to solve the delivery sequence of a Travel Sales Person (TSP) problem.
An online delivery company allows it's customers to state their prefered delivery time window.
This model optimizes the route of the delivery man/men to ensure all orders are delivered within customer's time-window while also minimizing the work hours and number of the delivery men.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from gurobipy import Model, GRB, quicksum

#import the excel Sheet containing coordinates of customers
workbook = load_workbook(filename = "customer details.xlsx")
sheet = workbook['Sheet1']
import math

In [2]:
#Extract the details for each customer and time windows if availale
n=25 #number of customers
v=2 #number of delivery men
d=10 #service duration  for each customer
driving_speed=20  #km/hr
depot=[sheet.cell(2,2).value,sheet.cell(2,3).value]
#coordinates of Depot and customers
lat=[depot[0]]
lon=[depot[1]]
for i in range(3,2+n+1):
    lat.append(sheet.cell(i,2).value)
    lon.append(sheet.cell(i,3).value)
    
#Time_window of all Customers 
time_window={i-2: [sheet.cell(i,4).value,sheet.cell(i,5).value,d] for i in range(3,2+n+1)}

time_window


{1: [540, 590, 10],
 2: [623, 673, 10],
 3: [566, 616, 10],
 4: [804, 854, 10],
 5: [883, 933, 10],
 6: [706, 756, 10],
 7: [644, 694, 10],
 8: [841, 891, 10],
 9: [882, 932, 10],
 10: [704, 754, 10],
 11: [773, 823, 10],
 12: [593, 643, 10],
 13: [854, 904, 10],
 14: [564, 614, 10],
 15: [563, 613, 10],
 16: [707, 757, 10],
 17: [855, 905, 10],
 18: [554, 604, 10],
 19: [551, 601, 10],
 20: [599, 649, 10],
 21: [607, 657, 10],
 22: [752, 802, 10],
 23: [870, 920, 10],
 24: [720, 770, 10],
 25: [657, 707, 10]}

In [9]:
v=1
c=n

In [11]:
#function to calculate distance between 2 nodes
def distance(node_from,node_to):
    return ((xc[node_from]-xc[node_to])**2 + (yc[node_from]-yc[node_to])**2)**0.5

#Adds the starting location of each delivery man as the depot. Also enables the code to be adjustable for more delivery men
for vehicle in range(v):
    lat.append(depot[0])
    lon.append(depot[1])
      
#Time(mins) between nodes is calculated as Haversine distance divided by speed
def time(node_from,node_to):
    driving_speed=20
    lat1,lon1=lat[node_from],lon[node_from]
    lat2,lon2=lat[node_to],lon[node_to]
    r=6371
    dlat=math.radians(lat2-lat1)
    dlon=math.radians(lon2-lon1)
    a=math.sin(dlat/2)**2 +math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)**2
    c=2*math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance=r*c
    time_minute=distance/driving_speed *60 #in minutes
    
    return time_minute




#N is set of customers
N=[i for i in range(1,n+1)]
#V is the set of nodes including the vehicles(deliverymen)
Vn=[i for i in range(1,v+n+1)]
#A is the set of Arcs between customers
A=[(i,j) for i in N for j in N ]
#AN is set of all Arcs including those of vehicles(deliverymen)
AN=[(i,j) for i in Vn for j in Vn ]
#t is the time of each arc including those from depot to customers
t={(i,j): np.round(time(i,j)) for i,j in AN}

In [3]:
#Define the optimization model stating the constraints and objectives

mdl=Model(name='TSPTW')
#Defining Variables
# xij variables are created with Constraint (5) from model
x=mdl.addVars(AN, vtype=GRB.BINARY)
#s for starting_time variables and st for Position in TSP journey of customer i
s=mdl.addVars(N,vtype=GRB.CONTINUOUS)
st=mdl.addVars(N,vtype=GRB.CONTINUOUS)
#Objective function (1)
mdl.modelSense=GRB.MINIMIZE
mdl.setObjective(quicksum(x[i,j]*t[i,j] for i,j in AN ))
#Constraints
#All Xij columns and rows equal 0   (2) and (3)
mdl.addConstrs(quicksum(x[i,j] for j in Vn if i!=j )==1 for i in Vn if i!=0 )
mdl.addConstrs(quicksum(x[i,j] for i in Vn if i!=j)==1 for j in Vn if i!=0)

#Time Window Constraint
# constraint(7)
def TW_constraint(i,j,x=x,time_window=time_window):
    return (s[i]-s[j] +time_window[i][2]+t[i,j]-(1-x[i,j])*(time_window[i][1]-time_window[j][0]+time_window[i][2]+t[i,j]))
mdl.addConstrs( TW_constraint(i,j)<=0 for i,j in A )
#constraint (8)
mdl.addConstrs(s[i]>=time_window[i][0] for i in N )
mdl.addConstrs(s[i]<=time_window[i][1] for i in N );
#SubTour Elimination Constraint
#constraints (6) and (4) respectively
mdl.addConstrs(st[i]-st[j]+1-n+n*x[i,j]<=0 for i,j in A )
mdl.addConstrs(st[i]>=1 for i in N )
mdl.addConstrs(st[i]<=n for i in N )


lab=mdl.addVars(N,vtype=GRB.CONTINUOUS)
for i in range(1,v+1):
    lab[n+i]=i
mdl.addConstrs(lab[i]-lab[j]-(1-x[i,j])*(v-1)<=0 for i,j in AN)

#mdl.Params.MIPGap=0.2
mdl.optimize()


Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2133 rows, 804 columns and 7183 nonzeros
Model fingerprint: 0x4007157e
Variable types: 75 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+02]
Presolve removed 1185 rows and 277 columns
Presolve time: 0.04s
Presolved: 948 rows, 527 columns, 7086 nonzeros
Variable types: 75 continuous, 452 integer (452 binary)

Root relaxation: objective 9.321763e+01, 207 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   93.21763    0   37          -   93.21763      -     -    0s
H    0     0                     131.0000000   93.21763  28.8%     -    0s
H    0     0       

In [2]:
#Printing out the solution and its Sequence
#We print the delivery sequence for each Vendor
vendor=[]
sequence=[]
#create a list of all nodes per vendor using the label lab for each node
for j in range(1,v+1):
    vendor.append({i:([np.round(st[i].x), np.round(s[i].x),np.round(s[i].x) + time_window[i][2] ]) for i in N if np.round(lab[i].x)==j})
    sequence.append([])

h=0
for delivery_man in vendor:
    for i in delivery_man:
        sequence[h].append((delivery_man[i],i))
        sequence[h].sort()
    h+=1
h=1
sequence_print='Depot '
for i in sequence:
    print(f"'Customer': 'Position','Visitation Time','Visitation Ending'")
    for j in range(len(i)):
        print(i[j][1],[j+1,i[j][0][1],i[j][0][2]])
    journey_start=int(np.round(i[0][0][1]-time(i[0][1],0)))

    #visitation time of last customer+ service duration +time back to depot
    journey_end=int(np.round(i[j][0][1]+time_window[i[j][1]][2])+time(0,i[j][1]))

    for j in range(len(i)):
        sequence_print+=f'-> Customer {i[j][1]}'
    sequence_print=sequence_print+' -> Depot'
    print(f'Delivery man {h} Sequence of delivery is : {sequence_print}\n')
    

    Total_time_spent_at_work=journey_end-journey_start
    print(f'Work window of Delivery Man {h}')
    print(f'''Work Start: {journey_start//60}:{journey_start%60}
    Work End : {journey_end//60}:{journey_end%60}
    Time spent at work (mins) {Total_time_spent_at_work}\n''')
    h+=1
print(f'Objective value:  {mdl.objval}, Program runtime(s) :{mdl.runtime}')

'Customer': 'Position','Visitation Time','Visitation Ending'
18 [1, 554.0, 564.0]
3 [2, 566.0, 576.0]
1 [3, 590.0, 600.0]
2 [4, 623.0, 633.0]
25 [5, 657.0, 667.0]
7 [6, 694.0, 704.0]
11 [7, 773.0, 783.0]
17 [8, 855.0, 865.0]
23 [9, 920.0, 930.0]
Delivery man 1 Sequence of delivery is : Depot -> Customer 18-> Customer 3-> Customer 1-> Customer 2-> Customer 25-> Customer 7-> Customer 11-> Customer 17-> Customer 23 -> Depot

Work window of Delivery Man 1
Work Start: 9:7
    Work End : 15:35
    Time spent at work (mins) 388

'Customer': 'Position','Visitation Time','Visitation Ending'
15 [1, 566.0, 576.0]
19 [2, 580.0, 590.0]
12 [3, 593.0, 603.0]
14 [4, 607.0, 617.0]
20 [5, 645.0, 655.0]
21 [6, 657.0, 667.0]
16 [7, 707.0, 717.0]
10 [8, 719.0, 729.0]
6 [9, 731.0, 741.0]
22 [10, 752.0, 762.0]
24 [11, 767.0, 777.0]
4 [12, 854.0, 864.0]
8 [13, 867.0, 877.0]
13 [14, 884.0, 894.0]
9 [15, 898.0, 908.0]
5 [16, 916.0, 926.0]
Delivery man 2 Sequence of delivery is : Depot -> Customer 18-> Customer 